## Stemmen voor alles behalve de stem
### Een visuele analyse van het stemgedrag bij het Eurovisie Songfestival

Ieder jaar, zo halverwege mei, verbindt het Eurovisie Songfestival muziekliefhebbers van over heel Europa. Het festival werd voor het eerst gehouden in 1956 met slechts zeven deelnemende landen. Vandaag de dag doen er elk jaar rond de veertig landen mee. Ieder land treedt op met een zelfgeschreven nummer, waarna er via stemming van een jury en televoting een winnaar wordt bepaald. 
Het land dat wint krijgt de eer om het jaar daarop het Eurovisie Songfestival te organiseren. Dat is voor een land een mooi moment van nation building, en het kan financiële voordelen bieden op de korte en lange termijn. Het winnen van het Eurovisie Songfestival is dus een gewichtige zaak. Daarom is het interessant om te weten welke factoren het meeste effect hebben op het stemgedrag tijdens het Eurovisie Songfestival.
In de eerste instantie zou je verwachten dat stemmen uitsluitend worden uitgedeeld op basis van muzikale voorkeur. Maar het lijkt alsof er meer meespeelt dan alleen dat. Door sommigen wordt opgemerkt dat stemgedrag tijdens het Eurovisie Songfestival meer afhankelijk is van culturele, politieke of willekeurige factoren dan van pure muzikale voorkeur. Opvallend is bijvoorbeeld dat sommige landen gemiddeld een stuk meer punten halen dan andere landen, zoals te zien is in figuur 1. En in 2022 heeft Oekraïne volgens sommigen enkel gewonnen door 'sympathy votes', vanwege de Russische invasie (Gilbert, 2022). Om dit soort biases te voorkomen, stemt elk land ook via een vakjury. Die is echter ook gevoelig gebleken voor dezelfde biases. De vraag is wat voor biases dit zijn. 

 Ieder land reikt punten uit waarbij 50 procent via televoting en 50 procent via een jury bepaald worden. Tot 2016 gaf een land 1 tot en met 8, 10 en 12 punten aan andere landen. Vanaf 2016 werden dit twee sets, één namens de jury en één namens televoting. In onderstaand figuur is te zien hoe de jury votes die een land per jaar krijgt gemiddeld ontwikkelen. Soms krijgt een land eenmalig meer punten, soms zit er een stijgende of dalende lijn in. In dit data story proberen we te achterhalen waar dat door komt, anders dan dat de liedjes goed zijn.


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import csv
import numpy as np

color_blind_red, color_blind_green, color_blind_blue = '#d95f02', '#1b9e77', '#7570b3'

In [2]:
# Importeer het dataframe met de punten die gegeven zijn per songfestival
votes_df = pd.read_csv('eurovision_song_contest_1975_2019.csv')
votes_df.columns=[column.strip() for column in votes_df.columns] 
votes_df = votes_df.loc[votes_df['Jury or Televoting'] == 'J']
votes_df = votes_df.drop(columns=['Edition', 'Duplicate', 'Jury or Televoting'])
votes_df.replace({'The Netherands': 'The Netherlands', 'F.Y.R. Macedonia':'North Macedonia', 'Macedonia':'North Macedonia'}, inplace=True)

# Seksualiteit dataframe inladen en opschonen
sexuality_df = pd.read_csv("Seksualiteit.csv", encoding= 'unicode_escape', header=None)
sexuality_header = ['Name', 'Country', 'Year', 'Song title', 'Sexuality', 'Points', 'Place']
sexuality_df.columns = sexuality_header  # Header toevoegen
sexuality_df['Country'] = sexuality_df['Country'].apply(str.strip)
sexuality_df = sexuality_df.drop(columns=['Name', 'Song title', 'Points', 'Place'])
sexuality_df.replace({'Netherlands': 'The Netherlands', 'Bosnia and Herzegovina': 'Bosnia & Herzegovina'}, inplace=True)

# gender database uitlezen en opschonen
gender_df = pd.read_csv('song_data.csv', sep=',', encoding= 'unicode_escape')
gender_df = gender_df.drop(columns=['race','host_10','age','semi_draw_position', 'final_draw_position', 'artist_name', 'song_name', 'language', 'style', 'direct_qualifier_10', 'main_singers', 'selection', 'key', 'BPM', 'energy', 'danceability', 'happiness', 'loudness', 'acousticness', 'instrumentalness', 'liveness', 'speechiness', 'release_date', 'key_change_10', 'backing_dancers', 'backing_singers', 'backing_instruments', 'instrument_10', 'qualified', 'final_televote_points', 'final_jury_points', 'final_televote_votes', 'final_jury_votes', 'final_place', 'final_total_points', 'semi_place', 'semi_televote_points', 'semi_jury_points', 'semi_total_points', 'favourite_10']) # onnodige kolommen verwijderen
gender_df.replace({'Netherlands': 'The Netherlands', 'Bosnia and Herzegovina':'Bosnia & Herzegovina'}, inplace=True)
gender_df['gender'] = gender_df['gender'].replace("Female", 'Vrouw')
gender_df['gender'] = gender_df['gender'].replace("Male", 'Man')

# oostblok landen dataframe uitlezen en opschonen
oostblok_df = pd.read_csv("oostblok.csv")
oostblok_df.replace({'Czechia': 'Czech Republic', 'Türkiye':'Turkey'}, inplace=True)

# lijst met landen die ooit mee hebben gedaan inladen
with open('landen_die_ooit_meededen.txt') as landen_file:
    landen = landen_file.readlines()
    landen = [land.strip('\n') for land in landen]
    landen = [land.strip('\ufeff') for land in landen]

# iso codes vertalen naar normale namen doormiddel van een csv bestand
iso_to_country_name = {}
with open('cc3_cn.csv', 'r', newline='') as iso_to_country_name_file:
    reader = csv.reader(iso_to_country_name_file)
    next(reader)
    for row in reader:
        iso_code, country_name = row
        iso_to_country_name[iso_code] = country_name

# land info database inladen
info_land_df = pd.read_csv('dist_cepii.csv', sep=';')
info_land_df = info_land_df.replace({"iso_o": iso_to_country_name} | {"iso_d": iso_to_country_name}) # vertalen van iso codes naar normale namen
info_land_df = info_land_df[info_land_df['iso_o'].isin(landen)] # filteren op landen die mee hebben gedaan met eurovision
info_land_df = info_land_df[info_land_df['iso_d'].isin(landen)]
info_land_df['contig'] = info_land_df['contig'].replace({1:True, 0:False})
info_land_df['comlang_off'] = info_land_df['comlang_off'].replace({1:True, 0:False})
info_land_df = info_land_df.drop(columns=['colony', 'comcol', 'curcol', 'col45', 'smctry', 'distcap', 'distw', 'distwces', 'comlang_ethno', 'dist'])

# voeg de dataframes samen gebaseerd op de kolommen die de landen en jaren aangeven
taal_votes_df = pd.merge(votes_df, info_land_df, left_on=['From country', 'To country'], right_on=['iso_o', 'iso_d'], how='left')
taal_votes_df = taal_votes_df.drop(columns=['iso_o', 'iso_d'])

taal_gender_votes_df = pd.merge(taal_votes_df, gender_df, left_on=['Year', 'To country'], right_on=['year', 'country'], how='left')
taal_gender_votes_df = taal_gender_votes_df.drop(columns=['year', 'country'])

taal_gender_sexuality_votes_df = pd.merge(taal_gender_votes_df, sexuality_df, left_on=['Year', 'To country'], right_on=['Year', 'Country'], how='left')
taal_gender_sexuality_votes_df['Sexuality'].loc[(taal_gender_sexuality_votes_df['Sexuality'].isnull() == False)] = 'Queer' 
taal_gender_sexuality_votes_df['Sexuality'].fillna('Hetero', inplace=True)
taal_gender_sexuality_votes_df = taal_gender_sexuality_votes_df.drop(columns=['Country'])

taal_gender_sexuality_oostblok_votes_df = pd.merge(taal_gender_sexuality_votes_df, oostblok_df, left_on=['From country'], right_on=['Land'], how='left')
taal_gender_sexuality_oostblok_votes_df.rename(columns={'Oostblok': 'From-oostblok'}, inplace=True)
taal_gender_sexuality_oostblok_votes_df = taal_gender_sexuality_oostblok_votes_df.drop(columns=['Land'])
taal_gender_sexuality_oostblok_votes_df = pd.merge(taal_gender_sexuality_oostblok_votes_df, oostblok_df, left_on=['To country'], right_on=['Land'], how='left')
taal_gender_sexuality_oostblok_votes_df.rename(columns={'Oostblok': 'To-oostblok'}, inplace=True)
taal_gender_sexuality_oostblok_votes_df = taal_gender_sexuality_oostblok_votes_df.drop(columns=['Land'])

new_header = ['Year', '(semi-) final', 'From country', 'To country', 'Points', 'Contigent', 'Common language', 'Gender', 'Sexuality',  'From-oostblok', 'To-oostblok']
taal_gender_sexuality_oostblok_votes_df.columns = new_header

deelnemer_info_df = pd.read_csv('contestants.csv')

# Bereken het gemiddeld aantal punten voor plek 1 tot en met 25 in de finale
deelnemer_info_df = deelnemer_info_df[deelnemer_info_df['running_final'] < 26]
running_place_points_df = deelnemer_info_df.groupby('running_final')['place_final'].mean()

eurovision_points_df = taal_gender_sexuality_oostblok_votes_df

/tmp/ipykernel_59068/1388124699.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taal_gender_sexuality_votes_df['Sexuality'].loc[(taal_gender_sexuality_votes_df['Sexuality'].isnull() == False)] = 'Queer'


In [3]:
# Kaart van de ontwikkeling van het gemiddelde van de punten in de finale per land
def total_per_year(dataframe):
    """
    Berekent de totale hoeveelheid punten die een land per jaar in de finale van het eurovisie
    van de vakjury heeft gekregen.
    """

    # Maak een lege dictionary aan voor het opslaan van de data
    total_year = {}
    
    # Itereer over de dataframe met gegeven punten
    for index, row in dataframe.iterrows():

        # Controleer of er een finale gezongen werd
        if row[1] == 'f':

            # Controleer of het land al in het dictionary staat
            if row[3] in total_year:

                # Controleer of het jaar al bij het land staat
                if row[0] in total_year[row[3]]:

                    # Als het jaar al aanwezig is bij het land, worden de punten opgeteld
                    total_year[row[3]][row[0]] += row[4]

                # Als het jaar nog niet aanwezig is wordt het jaar toegevoegd aan het dictionary
                # van het land
                else:
                    total_year[row[3]].update({row[0]: row[4]})

            # Als het land nog niet in het dictionary staat, wordt een nieuwe key aangemaakt met
            # daarin weer een dictionary van het jaar en het aantal punten dat behaald is
            else:
                total_year.update({row[3]: {row[0]:row[4]}})

    # Maak een dataframe om het dictionary in te kunnen zetten
    total_per_year_df = pd.DataFrame({'Country':[], 'Year':[], 'Points':[]})

    # Voeg alle elementen van het dictionary per land toe aan het dataframe
    for country, eurovision_points_df in total_year.items():
        for year, score in eurovision_points_df.items():
            total_per_year_df.loc[len(total_per_year_df)] = [country, year, score]

    # Sorteer het dataframe zodanig dat er slices per land gesorteerd op jaar ontstaan
    # Dit garandeert goed verloop van de functie voor de gemiddeldes
    sorted_df = total_per_year_df.sort_values(['Country' , 'Year'], ascending=[True, True], ignore_index=True)

    return sorted_df

def development_of_mean(dataframe):
    """
    Berekent de verandering van het gemiddelde totale aantal punten dat een land per jaar heeft gekregen.
    Elk jaar wordt dus het gemiddelde geupdate naar het nieuwe aantal punten dat is behaald, tenzij
    er niet is deelgenomen aan de finale. In dat geval wordt hetzelfde gemiddelde nog eens toegevoegd
    om gaten in de uiteindelijke grafiek te voorkomen. De functie vult ook de waardes aan tot het jaar
    2019, zodat de grafiek bij het eindpunt van de animatie alle uiteindelijke gemiddeldes bevat.
    """

    # Creëer een dataframe met de totale aantal punten per land per jaar
    total_per_year_df = total_per_year(dataframe)
    
    # Maak een dictionary met het eerste land, jaar en punten 
    mean_per_year = {total_per_year_df.iloc[0,0] : {total_per_year_df.iloc[0,1]: total_per_year_df.iloc[0,2]}}

    # Maak counters om de gemiddelden mee te kunnen berekenen
    total_years = 0
    total_points = total_per_year_df.iloc[0,2]

    # Loop over het dataframe met totale aantal punten, sla de eerste rij over
    for index, row in total_per_year_df[1:].iterrows():
        # Controleer of het land hetzelfde is als in de rij ervoor
        if row[0] == total_per_year_df.iloc[index - 1, 0]:

            # Als het land hetzelfde is, controleer of het jaar maar één verschilt
            # Zo niet, vul dan de missende jaren aan met het bestaande gemmiddelde
            if row[1] != (total_per_year_df.iloc[index - 1, 1] + 1):
                for year in range(total_per_year_df.iloc[index - 1, 1] + 1, row[1]):
                    mean_per_year[row[0]].update({year : (total_points / total_years)})

            # Vul het dict aan met het jaar en het nieuwe gemiddelde
            total_points += row[2]
            total_years += 1
            mean_per_year[row[0]].update({row[1] : (total_points / total_years)})        

        else:

            # Als het land niet hetzelfde is als de rij ervoor, controleer of het laatst aangevulde jaar 2019 is
            # Zo niet, vul dan het dict aan met hetzelfde gemiddelde tot 2019
            if total_per_year_df.iloc[index - 1, 1] != 2019:
                for year in range(total_per_year_df.iloc[index - 1, 1] + 1, 2020):
                    mean_per_year[total_per_year_df.iloc[index - 1, 0]].update({year : (total_points / total_years)})

            # Update het dict met het nieuwe land, jaar en aantal punten
            mean_per_year.update({row[0]: {row[1] : row[2]}})

            # Zet het totale aantal punten naar het ereste aantal punten en zet het aantal jaren op 1
            total_points = row[2]
            total_years = 1

    # Maak een dataframe om het dictionary in te kunnen zetten
    mean_per_year_df = pd.DataFrame({'Country':[], 'Year':[], 'Points':[]})

    # Itereer over het dictionary om de waardes in het dataframe te zetten
    for country, euro in mean_per_year.items():
        for year, score in euro.items():
            mean_per_year_df.loc[len(mean_per_year_df)] = [country, year, score]

    return mean_per_year_df

# Creëer een dataframe met het gemiddelde aantal punten per jaar
mean_per_year = development_of_mean(eurovision_points_df)

year_order = [year for year in range(1957, 2020)]

# Creëer een figuur met een wereldkaart waar per jaar de gemiddelde punten op worden laten zien
fig_per_year = px.choropleth(mean_per_year, 
                    category_orders={'Year': year_order},
                    locationmode='country names', 
                    locations='Country', 
                    color='Points',
                    animation_frame='Year',
                    title='Gemiddelde van totale aantal punten in de finale per land per jaar',
                    height=700,
                    color_continuous_scale=['#1E88E5', '#FFC107', '#FF005D'],
                    labels={'Country': 'Land', 'Year': 'Jaar', 'Points': 'Gemiddeld aantal punten'})

fig_per_year.show()

Figuur 1: Een geoplot die het gemiddeld gehaalde aantal punten van landen in de finale weergeeft per jaar. Met de slider kan per jaar bekeken worden hoe het gemiddelde tot dan toe is ontwikkeld. Indien een land in een jaar niet heeft meegedaan in de finale, blijft het gemiddelde hetzelfde.

Als we naar de onderstaande figuur kijken is te zien dat er clusters van landen zijn die vaak op elkaar stemmen. Toch zijn er ook clusters waar dit niet gebeurt. Opvallend is dat er 'voting blocks' zijn. Dat zijn clusters van landen die elk jaar weer veel op elkaar stemmen. In figuur 2 kun je zien dat sommige landen elkaar een stuk meer punten geven dan andere landen. Het gemiddelde aantal punten is ongeveer 2,7, terwijl de Balkan gemiddeld soms wel 9 of 10 punten aan elkaar geeft. 

In [4]:
east_block = sorted(['Belarus', 'Bulgaria', 'Czech Republic', 'Hungary', 'Poland', 'Moldova', 'Romania', 'Russia', 'Slovakia','Ukraine'])
west_block = sorted(['Belgium', 'France', 'Germany', 'Luxembourg', 'Austria', 'Monaco', 'The Netherlands','Switzerland', 'United Kingdom', 'Ireland'])
southern_block = sorted(['Albania', 'Bosnia & Herzegovina', 'Croatia', 'Greece', 'Italy', 'Malta', 'Montenegro', 'North Macedonia', 'Portugal', 'San Marino', 'Serbia', 'Slovenia', 'Spain'])
scandinavia = sorted(['Denmark', 'Finland', 'Iceland', 'Norway', 'Sweden'])
baltic_states = sorted(['Estonia', 'Latvia', 'Lithuania'])
benelux = sorted(['The Netherlands', 'Belgium', 'Luxembourg'])
balkan_states = sorted(['Albania', 'Bosnia & Herzegovina', 'Bulgaria', 'Greece', 'Montenegro', 'North Macedonia', 'Croatia', 'Serbia', 'Serbia & Montenegro'])
outside_europe = sorted(['Cyprus', 'Armenia', 'Australia', 'Israel', 'Azerbaijan'])



landnaam_vertaling = {}
with open('landen_vertaling.csv', 'r', newline='') as landnaam_vertaling_bestand:
    reader = csv.reader(landnaam_vertaling_bestand)
    for row in reader:
        engelsenaam, nederlandsenaam = row
        landnaam_vertaling[engelsenaam] = nederlandsenaam

def vertaal_landnamen(lijst):
    nieuwe_lijst = []
    for country in lijst:
        nieuwe_lijst.append(landnaam_vertaling[country])
    return nieuwe_lijst


def mean_points_from(countries):
    list_of_lists_points = []
    for from_country in countries:
        list_points = []
        for to_country in countries:
            if from_country == to_country:
                list_points.append(np.nan)
            else:
                points = eurovision_points_df.loc[(eurovision_points_df['From country'] == from_country) & (eurovision_points_df['To country'] == to_country), 'Points'].mean()
                list_points.append(points)
        list_of_lists_points.append(list_points)
    return list_of_lists_points

fig = go.Figure()

fig.add_trace(go.Heatmap(
                   z=mean_points_from(balkan_states),
                   x=vertaal_landnamen(balkan_states),
                   y=vertaal_landnamen(balkan_states),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=False,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12
))

fig.add_trace(go.Heatmap(
                   z=mean_points_from(baltic_states),
                   x=vertaal_landnamen(baltic_states),
                   y=vertaal_landnamen(baltic_states),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=False,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12
))

fig.add_trace(go.Heatmap(
                   z=mean_points_from(benelux),
                   x=vertaal_landnamen(benelux),
                   y=vertaal_landnamen(benelux),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=True,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12
))

fig.add_trace(go.Heatmap(
                   z=mean_points_from(outside_europe),
                   x=vertaal_landnamen(outside_europe),
                   y=vertaal_landnamen(outside_europe),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=False,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12
))

fig.add_trace(go.Heatmap(
                   z=mean_points_from(east_block),
                   x=vertaal_landnamen(east_block),
                   y=vertaal_landnamen(east_block),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=False,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12
))

fig.add_trace(go.Heatmap(
                   z=mean_points_from(scandinavia),
                   x=vertaal_landnamen(scandinavia),
                   y=vertaal_landnamen(scandinavia),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=False,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12
))

fig.add_trace(go.Heatmap(
                   z=mean_points_from(west_block),
                   x=vertaal_landnamen(west_block),
                   y=vertaal_landnamen(west_block),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=False,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12
))

fig.add_trace(go.Heatmap(
                   z=mean_points_from(southern_block),
                   x=vertaal_landnamen(southern_block),
                   y=vertaal_landnamen(southern_block),
                   colorscale=['#1E88E5', '#FFC107', '#FF005D'],
                   visible=False,
                   colorbar=dict(len=1, y=0.5),
                   zmin=0,
                   zmax=12

))

fig.update_layout(
    xaxis=dict(
        scaleanchor='y',
        scaleratio=1,
        title='Land dat punten ontvangt',
        automargin=False,
        title_standoff=225,
        tickangle=45
    ),
    yaxis=dict(
        constrain='domain',
        title='Land dat punten geeft',
        automargin=False,
        title_standoff=225,
        tickangle=0
        ),
    autosize=False,
    margin=dict(t=50, b=150, l=150, r=0),
    height=575,
    width=800
)


fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{"visible": [False, False, True, False, False, False, False, False]}],
                    label="Benelux",
                    method="update"
                ),
                dict(
                    args=[{"visible": [True, False, False, False, False, False, False, False]}],
                    label="Balkan",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, True, False, False, False, False, False, False]}],
                    label="Baltische staten",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, False, False, True, False, False, False, False]}],
                    label="Niet Europese landen",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, False, False, False, True, False, False, False]}],
                    label="Oostblok",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, False, False, False, False, True, False, False]}],
                    label="Scandinavië",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, False, False, False, False, False, True, False]}],
                    label="West Europa",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, False, False, False, False, False, False, True]}],
                    label="Zuid Europa",
                    method="update"
                ),
            ]),
            type='buttons',
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=1.3,
            xanchor="left",
            y=.9,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
    title_text="Gemiddelde hoeveelheid punten gegeven tussen landen"
)

fig.show()

Figuur 2: Heatmap die voor verschillende clusters van landen de gemiddelde hoeveelheid punten die landen aan elkaar geven weergeeft. Met de knoppen aan de zijkant kan een cluster gekozen worden. Er is in sommige clusters symmetrie te herkennen, wat betekent dat de landen veelal dezelfde punten aan elkaar geven. 

Dit fenomeen heeft waarschijnlijk een culturele oorzaak. Het kan zijn dat mensen meer geneigd zijn om te stemmen op landen die een culturele gelijkenis hebben met hun eigen land. Dat is niet zo vreemd, want culturele gelijkenis kan ervoor zorgen dat de liedjes beter aansluiten bij hun belevingswereld of voorkeuren. Om aan te tonen dat het stemgedrag van landen beïnvloed wordt door culturele gelijkenissen tussen landen, is het effect van drie belangrijke culturele aspecten onderzocht. Dit zijn gelijkenis van de taal, gelijkenis van de geografische locatie en gelijkenis van de politieke geschiedenis. 

Aan de andere kant lijkt het alsof stemgedrag vooral wordt beïnvloed door willekeurige factoren die eigenlijk irrelevant zijn voor de muziek zelf. Om dat aan te tonen, is gekeken naar het effect van de volgorde van de optredens en de verdeling van de halve finales. Daarnaast is gekeken naar het effect van gender en seksuele oriëntatie van de winnende artiesten.


## Culturele factoren
In het eerste perspectief wordt dus gekeken naar culturele factoren zoals taal en naburigheid van landen. 

In [5]:
# berekenen van gemiddelde en mediaan voor landen die wel of niet dezelfde taal hebben en wel of niet aanliggend zijn
mean_contig = eurovision_points_df[eurovision_points_df['Contigent']==1]['Points'].mean()
mean_non_contig = eurovision_points_df[eurovision_points_df['Contigent']==0]['Points'].mean()
median_contig = eurovision_points_df[eurovision_points_df['Contigent']==1]['Points'].median()
median_non_contig =eurovision_points_df[eurovision_points_df['Contigent']==0]['Points'].median()

mean_comlang = eurovision_points_df[eurovision_points_df['Common language']==1]['Points'].mean()
mean_non_comlang = eurovision_points_df[eurovision_points_df['Common language']==0]['Points'].mean()
median_comlang = eurovision_points_df[eurovision_points_df['Common language']==1]['Points'].median()
median_non_comlang = eurovision_points_df[eurovision_points_df['Common language']==0]['Points'].median()

fig = go.Figure()

# maak alle individuele grafieken
fig.add_trace(
    go.Bar(name='Waar',
           x=['Naburig', 'Zelfde taal'],
           y=[mean_contig, mean_comlang],
           visible=True,
           marker=dict(color=color_blind_green))
)

fig.add_trace(
    go.Bar(name='Niet waar',
           x=['Naburig', 'Zelfde taal'],
           y=[mean_non_contig, mean_non_comlang],
           visible=True,
           marker=dict(color=color_blind_red))
)

fig.add_trace(
    go.Bar(name='Waar',
           x=['Naburig', 'Zelfde taal'],
           y=[median_contig, median_comlang],
           visible=False,
           marker=dict(color=color_blind_green))
)

fig.add_trace(
    go.Bar(name='Niet waar',
           x=['Naburig', 'Zelfde taal'],
           y=[median_non_contig, median_non_comlang],
           visible=False,
           marker=dict(color=color_blind_red))
)

# update de layout van algemene grafiek
fig.update_layout(
    width=600,
    height=700,
    autosize=False,
    margin=dict(t=40, b=20, l=0, r=0),
    template="plotly_white",
    title='Gemiddelde hoeveelheid punten gegeven door landen <br> die naburig zijn of dezelfde taal hebben',
    yaxis=dict(
        title="Gemiddelde hoeveelheid gegeven punten"
            )
)

# voeg een dropdown box toe waar kan worden gekozen tussen de paren van grafieken
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{"visible": [True, True, False, False]}, {"yaxis.title.text": "Gemiddelde hoeveelheid gegeven punten"}],
                    label="Gemiddelde",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, False, True, True]}, {"yaxis.title.text": "Mediaan van de gegeven punten"}],
                    label="Mediaan",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=1.05,
            xanchor="left",
            y=.9,
            yanchor="top"
        ),
    ]
)
fig.show()

Figuur 3: Bar chart waar de gebruiker kan kiezen of ze de mediaan of het gemiddelde willen zien van de gemiddelde hoeveelheid punten die landen aan elkaar geven, verdeeld op gemeenschappelijke taal en naburigheid.

Figuur 3 toont aan dat landen die eenzelfde officiële taal delen, elkaar ongeveer twee keer zo veel punten toekennen. Aangezien taal een cultureel goed is toont deze correlatie aan dat culturele gelijkenis een positieve invloed heeft op het aantal toegekende punten tussen landen onderling.  Uit het figuur blijkt ook dat naburige landen meer op elkaar stemmen dan op niet-aangrenzende landen. Naburigheid is een indicatie van een culturele overeenkomst. Deze correlatie geeft aan dat culturele gelijkenis een positief effect heeft op het toekennen van punten. 

In [6]:
nob_nob_points_list = []
nob_ob_points_list = []
ob_ob_points_list = []
ob_nob_points_list = []

for year in range(1975, 2019):   
    nob_to_nob = eurovision_points_df[(eurovision_points_df['From-oostblok'] == False) & (eurovision_points_df['To-oostblok'] ==False) & (eurovision_points_df["Year"] ==year)]["Points"]
    nob_nob_points = nob_to_nob.sum()

    nob_to_ob = eurovision_points_df[(eurovision_points_df['From-oostblok'] == False) & (eurovision_points_df['To-oostblok'] == True) & (eurovision_points_df["Year"] ==year)]["Points"]
    nob_ob_points = nob_to_ob.sum()

    ob_to_ob = eurovision_points_df[(eurovision_points_df['From-oostblok'] == True) & (eurovision_points_df['To-oostblok'] == True) & (eurovision_points_df["Year"] ==year)]["Points"]
    ob_ob_points = ob_to_ob.sum()

    ob_to_nob = eurovision_points_df[(eurovision_points_df['From-oostblok'] == True) & (eurovision_points_df['To-oostblok'] ==False) & (eurovision_points_df["Year"] ==year)]["Points"]
    ob_nob_points = ob_to_nob.sum()

    nob_nob_points_list.append(nob_nob_points)
    nob_ob_points_list.append(nob_ob_points)
    ob_ob_points_list.append(ob_ob_points)
    ob_nob_points_list.append(ob_nob_points)

trace1 = go.Scatter(x=list(range(1975, 2019)), y=nob_nob_points_list, mode='lines', name='Niet-Oostblok naar Niet-Oostblok', line=dict(color='blue'))
trace2 = go.Scatter(x=list(range(1975, 2019)), y=nob_ob_points_list, mode='lines', name='Niet-Oostblok naar Oostblok', line=dict(color='blue', dash='10'))
trace3 = go.Scatter(x=list(range(1975, 2019)), y=ob_ob_points_list, mode='lines', name='Oostblok naar Oostblok', line=dict(color='red', dash='10'))
trace4 = go.Scatter(x=list(range(1975, 2019)), y=ob_nob_points_list, mode='lines', name='Oostblok naar Niet-Oostblok', line=dict(color='red'))

# Create the data list
data = [trace1, trace2, trace3, trace4]

# Create the layout
layout = go.Layout(title='Stemgedrag Oostblok en Niet-Oostblok landen', 
                   xaxis=dict(title='Jaar'), 
                   yaxis=dict(title='Ontvangen punten'),
                   width=1000)

# Create the figure
ob_fig = go.Figure(data=data, layout=layout)

# Display the figure
ob_fig.show()


Figuur 4: Weergave van de jaarlijkse verdeling aantal stemmen van Oostblok en Niet-Oosblok landen aan zichzelf en elkaar. Op de y-as staan de totale ontvangen punten van dat jaar. Er is dus niet gecorrigeerd voor het aantal landen uit het oostblok dat in de finale heeft gestaan. In de grafiek is vaak te zien dat beide het Oostblok en het Niet-Oostblok op dezelfde partijen stemmen. De afwijking tussen twee lijnen met dezelfde trend kan veroorzaakt worden door de naburigheid van landen in dezelfde groep.

Om de gelijkheid van politieke geschiedenis te kunnen toetsen als variabele, hebben we gekeken naar het Oostblok. Landen die onderdeel zijn geweest van het Oostblok hebben een gelijke politieke geschiedenis gehad, of in ieder geval dezelfde politieke invloed. Figuur 4 toont aan dat het niet uitmaakt of er sprake is van een gemeenschappelijke politieke geschiedenis, omdat beide groepen vaak op hetzelfde gebied stemmen. Deze culturele variabele heeft dus geen impact op het stemgedrag van landen.

## Niet-culturele factoren
Overige factoren die niets met muziek te maken hebben zijn onder andere queerness van de artiest, of landen in dezelfde semi-final hebben gezeten en de volgorde waarin de liedjes worden gespeeld. Deze zijn onafhankelijk van het land en de cultuur waar de jury vandaan komt.

In [7]:
genders = ['Vrouw', 'Man']
sexualities = eurovision_points_df['Sexuality'].unique().tolist()
mean_values_dictionary = pd.DataFrame(columns=['Gender', 'Sexuality', 'Mean'])

for gender in genders:
    for sexuality in sexualities:
        list_of_points = eurovision_points_df.loc[(eurovision_points_df['Gender'] == gender) & (eurovision_points_df['Sexuality'] == sexuality), 'Points']
        new_row = pd.DataFrame({'Gender':[gender], 'Sexuality':[sexuality], 'Gemiddelde':[list_of_points.mean()]})
        mean_values_dictionary = pd.concat([mean_values_dictionary, new_row], ignore_index=True)

queerness = px.treemap(mean_values_dictionary, path=['Sexuality', 'Gender'], values='Gemiddelde',
                  color='Gemiddelde',
                  color_continuous_scale='RdBu_r')

queerness.update_layout(
    margin=dict(t=50, l=25, r=25, b=25),
    title="Behaalde punten door Queer en Hetero artiesten",
    annotations=[]
)

queerness.show()

Figuur 5: Treemap die weergeeft hoe veel punten Queer en Hetero mannen en vrouwen over alle jaren gemiddeld behaald hebben. Opvallend is dat Queer vrouwen aanzienlijk meer punten halen dan de andere identiteiten en oriëntaties.

Figuur 5 toont aan dat er meer punten toegekend worden aan landen met queer artiesten. Zes procent van alle deelnemers is openlijk queer, maar van alle winnaars is dit zestig procent. Hieruit blijkt dat er een positieve correlatie is tussen queerness van een artiest en het aantal stemmen dat dat land krijgt. 

In [10]:
details = {'Year': [], 'Punten gegeven <br>door landen uit': [], 'to': [], 'Percentages':[]}

for i in range(1975, 2020):
    candidates_sf1 = eurovision_points_df[(eurovision_points_df['Year'] == i) & (eurovision_points_df['(semi-) final'] == 'sf1')]['From country']
    sf1 = list(set(candidates_sf1))

    candidates_sf2 = eurovision_points_df[(eurovision_points_df['Year'] == i) & (eurovision_points_df['(semi-) final'] == 'sf2')]['From country']
    sf2 = list(set(candidates_sf2))

    sf1_to_sf1 = eurovision_points_df[(eurovision_points_df['From country'].isin(sf1)) & (eurovision_points_df['To country'].isin(sf1)) & (eurovision_points_df['Year'] == i) & (eurovision_points_df['(semi-) final'] == 'f')]['Points']
    s11 = sf1_to_sf1.sum()

    sf1_to_sf2 = eurovision_points_df[(eurovision_points_df['From country'].isin(sf1)) & (eurovision_points_df['To country'].isin(sf2)) & (eurovision_points_df['Year'] == i) & (eurovision_points_df['(semi-) final'] == 'f')]['Points']
    s12 = sf1_to_sf2.sum()

    sf2_to_sf2 = eurovision_points_df[(eurovision_points_df['From country'].isin(sf2)) & (eurovision_points_df['To country'].isin(sf2)) & (eurovision_points_df['Year'] == i) & (eurovision_points_df['(semi-) final'] == 'f')]['Points']
    s22 = sf2_to_sf2.sum()

    sf2_to_sf1 = eurovision_points_df[(eurovision_points_df['From country'].isin(sf2)) & (eurovision_points_df['To country'].isin(sf1)) & (eurovision_points_df['Year'] == i) & (eurovision_points_df['(semi-) final'] == 'f')]['Points']
    s21 = sf2_to_sf1.sum()
    if (sum([s11,s12,s21,s22])) != 0:
        all_points = sum([s11,s12,s21,s22])
        percentages = [s11/all_points*100, s12/all_points*100, s21/all_points*100, s22/all_points*100]

        details['Year'] += [i, i, i, i]
        details['Punten gegeven <br>door landen uit'] += ['T', 'T', 'F', 'F']
        details['to'] += ['T', 'F', 'T', 'F']
        details['Percentages'] += percentages

 
# creating a Dataframe object
df = pd.DataFrame(details)

mere_exp_fig = px.box(df, y='Percentages', 
             color='Punten gegeven <br>door landen uit', 
             facet_col='to', 
             facet_col_wrap=2, 
             title = 'Mere-exposure effect tussen de twee halve finales uit Eurovision')

# Update the layout with new subplot titles
mere_exp_fig.update_traces(
    legendgroup='Punten gegeven door',
    name='halve finale 1',
    selector=dict(name='T'),
    marker=dict(color=color_blind_green)
)

mere_exp_fig.update_traces(
    legendgroup='Punten gegeven door',
    name='halve finale 2',
    selector=dict(name='F'),
    marker=dict(color=color_blind_red)
)
mere_exp_fig.update_layout(
    annotations=[
        dict(
            x=0.18,
            y=1.02,
            xref='paper',
            yref='paper',
            text='Punten voor kandidaten uit de 1e halve finale',
            showarrow=False,
            font=dict(size=14)
        ),
        dict(
            x=0.68,
            y=1.02,
            xref='paper',
            yref='paper',
            text='<br>Punten voor kandidaten uit de 2e halve finale',
            showarrow=False,
            font=dict(size=14)
        )
    ]
)

mere_exp_fig.show()

Figuur 6: Boxplot die aantoont hoe de punten procentueel worden onderverdeeld van en naar landen uit de twee halve finales. De mediaan van de totale hoeveelheid punten die landen uit dezelfde halve finale aan elkaar hebben gegeven ligt zichtbaar hoger, wat betekent dat landen vaker stemmen op liedjes uit hun eigen halve finale.

Figuur 6 toont aan dat landen die aan dezelfde halve finale hebben deelgenomen elkaar meer punten geven. Dit kan onder andere komen door het exposure-effect, een psychologisch fenomeen dat stelt dat mensen een voorkeur ontwikkelen voor dingen die herkenbaarder zijn (Verrier, 2012). De kans is groot dat landen alleen naar hun eigen halve finale kijken (in plaats van allebei) en dus eerder geneigd zijn te stemmen op landen die ze vaker hebben gehoord. Dit blijkt ook uit figuur 6. Deze correlatie toont aan dat bij het stemmen factoren meespelen die noch op de muziek, noch op de cultuur betrekking hebben.

In [9]:
# Eigenschappen van de marker
marker = dict(
        color = running_place_points_df.values,
        colorscale = 'Portland_r',
        cmin = running_place_points_df.max(),
        cmax = running_place_points_df.min(),
        colorbar = dict(title='Eindplaats in de finale'),
        size = 15
    )

# Eigenschappen van de lijn
line = dict(
    color = 'grey',
    dash = 'dot'
   
)

# Maak een scattter plot van het gemiddeld aantal punten per afspeelplaats
afspeelplaats = go.Figure(data=go.Scatter(
    x = running_place_points_df.index,
    y = running_place_points_df.values,
    mode = 'lines + markers',
    marker = marker,
    line = line
))

# Plot het figuur met titel en labels
afspeelplaats.update_layout(title = 'Gemiddelde eindplaats van de afspeelplaatsen in de finale', 
                  xaxis_title='Afspeelplaats', 
                  yaxis_title='Gemiddelde eindplaats',
                  yaxis=dict(autorange='reversed'))

afspeelplaats.show()

Figuur 7: Lijngrafiek die aantoont waar de zangers gemiddeld eindigen per afspeelplaats in de finale. In de grafiek is er een opvallende dip bij de act en de acts na de pauze. 

Figuur 7 laat zien dat er grote verschillen zijn tussen de posities waarin nummers gemiddeld eindigen, afhankelijk van hun plek in het programma. Opvallend is dat nummers na de eerste twee posities steeds beter presteren. In het midden van het programma scoren landen opeens een stuk minder goed, maar dat herstelt zich snel. Richting het einde van het programma verslechtert de gemiddelde positie weer. 

Een mogelijke verklaring voor de stijging in de eerste helft is dat latere optredens beter worden onthouden, en die dus meer stemmen krijgen. De slechte resultaten in het midden kunnen komen doordat er meestal een pauze is tussen de twaalfde en de dertiende act en mensen later pas weer aansluiten om te kijken. De daling aan het einde kan komen door een afnemende interesse in het programma. De beste tijden om op te treden zijn in ieder geval vlak voor de pauze of een paar optredens na de pauze.


# Conclusie
Welk land de meeste kans heeft om het Eurovisie Songfestival te winnen, is dus afhankelijk van veel meer dan alleen de ingediende nummers. Aan de ene kant hebben de vaste factoren taal en naburigheid invloed. Landen kunnen aan deze factoren niets veranderen, maar deze wellicht wel benutten door erop in te spelen. Het kan bijvoorbeeld helpen om een nummer in de meest gesproken officiële taal van het land te schrijven, of in verschillende talen. Aan de andere kant spelen er factoren mee die elk jaar weer kunnen veranderen, zoals indeling van de semifinals, de programmering en de gender en seksualiteit van de artiesten. Landen hebben geen invloed op de programmering of de indeling van de semifinals en kunnen die factoren dus niet optimaliseren. Wel kunnen ze queer artiesten sturen, omdat die een grotere winkans hebben. 